# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Part 2: VacationPy

Now, let's use your skills working with weather data to plan future vacations. Use Jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** If you are having trouble displaying the maps, run `jupyter nbextension enable --py gmaps` in your environment and then retry.

To complete this part of the assignment, you will need to do the following:

* Create a heat map that displays the humidity for every city from Part 1, as in the following image:

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a reasonable number.

* Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap, with each pin containing the **Hotel Name**, **City**, and **Country**, as in the following image:

  ![hotel map](Images/hotel_map.png)

As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part 1, you must include a written description of three observable trends based on the data.
* For Part 2, you must take a screenshot of the heatmap that you create and include it in your submission.
* Your plots must include labeling aspects like plot title (with date of analysis) and axis labels.
* For max intensity in the heatmap, try setting it to the highest humidity found in the dataset.


In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
csvpath = '../weatherpy/output_data/weather_data.csv'
vacation_df = pd.read_csv(csvpath)
vacation_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        544 non-null    object 
 1   Lat         544 non-null    float64
 2   Lng         544 non-null    float64
 3   Max Temp    544 non-null    float64
 4   Humidity    544 non-null    int64  
 5   Cloudiness  544 non-null    int64  
 6   Wind Speed  544 non-null    float64
 7   Country     541 non-null    object 
 8   Date        544 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 38.4+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#determine max humidity level
max_hum = vacation_df['Humidity'].max()
max_hum

100

In [22]:
# Configure gmaps with API key:
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = vacation_df[['Lat', 'Lng']]


# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)

#plt heatmap
fig=gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_hum,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a reasonable number.

Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.

Plot the hotels on top of the humidity heatmap, with each pin containing the Hotel Name, City, and Country, as in the following image:

hotel map

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# Create new data frame with requested weather conditions:
vacation_custom=vacation_df[(vacation_df['Max Temp'] < 80) &
                             (vacation_df['Max Temp'] > 70) &
                             (vacation_df['Wind Speed'] < 10) &
                             (vacation_df['Cloudiness'] ==0)]
                            

vacation_custom.info() #no null data
vacation_custom

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 47 to 518
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        5 non-null      object 
 1   Lat         5 non-null      float64
 2   Lng         5 non-null      float64
 3   Max Temp    5 non-null      float64
 4   Humidity    5 non-null      int64  
 5   Cloudiness  5 non-null      int64  
 6   Wind Speed  5 non-null      float64
 7   Country     5 non-null      object 
 8   Date        5 non-null      int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 400.0+ bytes


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,Hilo,19.7297,-155.0900,77.29,73,0,6.91,US,1661783650
200,Saint George,37.1041,-113.5841,77.43,46,0,5.75,US,1661783958
402,Kununurra,-15.7667,128.7333,71.58,60,0,0.00,AU,1661783773
459,Alyangula,-13.8483,136.4192,72.59,90,0,7.00,AU,1661784076
518,Magalia,39.8121,-121.5783,76.12,35,0,4.00,US,1661784102


In [44]:
# Define hotel_df to store the hotel names 
hotel_df = vacation_custom

# params dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key }

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}" #add to a dictionary
    # Base url 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    response = requests.get(base_url, params=params)
    response_json = response.json()
    #print(response_json)
    hotel_df.loc[index, "Hotel Name"] =  response_json["results"][0]["name"] #add to hotel df
    
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,Hilo,19.7297,-155.0900,77.29,73,0,6.91,US,1661783650,Hilo Hawaiian Hotel
200,Saint George,37.1041,-113.5841,77.43,46,0,5.75,US,1661783958,Best Western Plus Abbey Inn
402,Kununurra,-15.7667,128.7333,71.58,60,0,0.00,AU,1661783773,Hotel Kununurra
459,Alyangula,-13.8483,136.4192,72.59,90,0,7.00,AU,1661784076,Groote Eylandt Lodge
518,Magalia,39.8121,-121.5783,76.12,35,0,4.00,US,1661784102,Paradise Pines RV Park - FHU - Safe & Clean


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
markers = gmaps.marker_layer(locations)
fig.add_layer(hotel_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))